<h1>Multiclass Classification</h1><br>
Es gibt dabei single-lable multiclass classification, bei der eine Beobachtung nur einer Klasse zugeordnet werden kann. Daraus folgt multi-lable ..., bei der eine Beobachtung zu mehreren Klassen gleichzeitig gehören kann.<br><br>
<b>Der Reuters-Datensatz</b><br>
Textklassifikation: kurze Nachrichten mit dem dazugehörigen Thema. Es gibt 46 Themen.

In [15]:
from keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words = 10000)
# es gilt wieder, dass jede Beobachtung nur aus den 10 000 häufigsten Wörtern besteht
print(len(train_data))
print(len(test_data))

8982
2246


Hier kann man wie bei der binären Klassifikation Daten in natürliche Sprache zurückumwandeln.

In [16]:
index = reuters.get_word_index()
print(index['car']) # assziativer Array, Wort -> Zahl 'Dictionary' in Python
# index.items() wandelt ein dict in Paare von zugeordneten Werten um
# und dict() wandelt Tupel wieder in dicts um
index_reversed = dict([(num, word) for (word, num) in index.items()])
# hat einen Offset von 3, weil die ersten 3 für "padding", "start" und "unknown" stehen
print(' '.join([index_reversed.get(i - 3, '?') for i in train_data[28]]))

1531
? oper shr 40 cts vs 30 cts oper net 1 364 000 vs 1 025 000 revs 14 7 mln vs 11 0 mln avg shrs 3 372 970 vs 3 425 400 year oper shr 86 cts vs 32 cts oper net 2 925 000 vs 1 109 000 revs 43 0 mln vs 35 7 mln avg shrs 3 383 651 vs 3 418 594 note year ago periods exclude extraordinary gain of 1 1 mln dlrs or 31 cts shr includes gains of 988 000 dlrs vs one mln dlrs in qtr and 2 2 mln dlrs vs 1 1 mln dlrs in year from tax loss carryforwards reuter 3


Nun geht es wieder ans Umwandeln der Daten in One-Hot-Kodierung für das NN.

In [17]:
import numpy as np

def one_hottify(newswire, num_classes):
    results = np.zeros((1, num_classes)).flatten()    
    for num in newswire:
        results[num] = 1        
    return results

news = train_data[0]
print(news, '\n')
print(one_hottify(news, 10000))
print(train_data)

[1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12] 

[0. 1. 1. ... 0. 0. 0.]
[list([1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12])
 list([1, 3267, 699, 3434, 2295, 56, 2, 7511, 9, 56, 3906, 1073, 81, 5, 1198, 57, 366, 737, 132, 20, 4093, 7, 2, 49, 2295, 2, 1037, 3267, 699, 3434, 8, 7, 10, 241, 16, 855, 129, 231, 783, 5, 4, 587, 2295, 2, 2, 775, 7, 48, 34, 191, 44, 35, 1795, 505

In [18]:
one_hot_train_data = [one_hottify(news, 10000) for news in train_data]
one_hot_test_data = [one_hottify(news, 10000) for news in test_data]
print(train_data[29], '\n')
print(one_hot_train_data[29])

[1, 2163, 317, 65, 131, 1462, 23, 768, 1225, 2, 7, 316, 5, 10, 3851, 1017, 97, 2201, 267, 2093, 248, 705, 9, 387, 262, 2, 4805, 118, 10, 163, 483, 36, 118, 4, 978, 427, 691, 491, 8140, 303, 522, 144, 34, 10, 2783, 5, 1236, 10, 1026, 24, 41, 8244, 4, 816, 168, 7372, 184, 75, 163, 126, 5, 4, 3579, 2164, 4805, 34, 3051, 6, 10, 1031, 515, 106, 909, 5, 65, 4274, 4805, 8, 144, 62, 7854, 5, 2, 696, 3851, 8034, 6, 65, 182, 4274, 64, 875, 548, 336, 7, 329, 206, 405, 439, 4, 825, 7, 4, 1142, 3813, 317, 6227, 101, 21, 2, 2552, 2988, 65, 1523, 254, 397, 5767, 6, 401, 20, 22, 193, 20, 9, 4, 5414, 5, 6227, 21, 4886, 65, 292, 54, 397, 36, 8, 4, 182, 250, 4, 8140, 40, 85, 2629, 13, 4, 3984, 5, 4169, 4274, 21, 65, 7, 809, 6, 842, 2247, 13, 9370, 112, 10, 7556, 5, 1490, 87, 9, 2163, 1577, 456, 691, 411, 184, 6223, 1381, 6, 1144, 226, 8211, 2, 6, 42, 414, 2828, 4274, 6476, 268, 1648, 8799, 6, 2563, 405, 6771, 5, 2, 8034, 984, 5, 6272, 756, 50, 2069, 106, 995, 389, 717, 9, 91, 325, 2069, 10, 1716, 5, 2512

Da es sich um ein Multiklassen-Problem handelt, müssen die Labels auch erst in One-Hot umgewandelt werden.

In [19]:
max_num = max(train_labels)
min_num = min(train_labels)
print(max_num, min_num) # Es gibt 46 Klassen
max_classes = max_num + 1

def label_one_hottify(num, max_classes):
    # has to deal with single numbers, not arrays
    res = np.zeros((1, max_classes)).flatten()
    res[num] = 1
    return res

one_hot_train_labels = [label_one_hottify(label, max_classes) for label in train_labels]
one_hot_test_labels = [label_one_hottify(label, max_classes) for label in test_labels]
print(train_labels[28], one_hot_train_labels[28])

45 0
3 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


<b>Bau des Netzwerks</b><br>
Es wichtig zu verstehen, dass jede Schicht ihre Informationen aus der vorangegangenen Schicht bezieht. Verlorengegangene Information ist nicht wiederherstellbar. Diese kann verloren gehen, indem z.B. eine Schicht weniger Knoten hat als die Outputschicht. Das heißt, diese Schicht stellt die Information des Inputs in weniger Knoten dar als die des Outputs und komprimiert Information über das geforderte Maß der letzten Schicht hinaus.

In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(64, activation = 'relu', input_shape = (10000, )))
model.add(layers.Dense(64, activation = 'relu'))
model.add()